In [1]:
import csv

import numpy as np
import rasterio
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [2]:
dataset = rasterio.open("./data/crs.tiff")
img_data = dataset.read(1)

In [3]:
with open('training/data_bin.csv', 'r') as f:
    reader = csv.DictReader(f)
    labeled_coords = [r for r in reader]
print("labeled dataset contains {} points".format(len(labeled_coords)))

labeled dataset contains 115 points


In [4]:
block_size = 28


def get_satellite_data(coord):
    x, y = dataset.index(float(coord['lon']), float(coord['lat']))
    img = img_data[x - block_size // 2:x + block_size // 2, y - block_size // 2:y + block_size // 2]
    img = img.astype(np.float32)
    img = img / 255
    img = np.expand_dims(img, -1)
    return img

In [5]:
X = list(map(get_satellite_data, labeled_coords))
Y = [int(l['windmill']) for l in labeled_coords]

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

X_train = np.stack(X_train)
X_test = np.stack(X_test)
Y_train = keras.utils.to_categorical(Y_train, 2)
Y_test = keras.utils.to_categorical(Y_test, 2)

In [7]:
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2021-11-06 12:25:57.907491: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-06 12:25:57.927747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2021-11-06 12:25:57.927763: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-11-06 12:25:57.928506: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [8]:
model.fit(X_train, Y_train, epochs=5)

2021-11-06 12:25:58.871694: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
3/3 [==============================] - 0s 7ms/step - loss: 0.6964 - accuracy: 0.5375
Epoch 2/5
3/3 [==============================] - 0s 7ms/step - loss: 0.6141 - accuracy: 0.6375
Epoch 3/5
3/3 [==============================] - 0s 8ms/step - loss: 0.6166 - accuracy: 0.6500
Epoch 4/5
3/3 [==============================] - 0s 8ms/step - loss: 0.5712 - accuracy: 0.7375
Epoch 5/5
3/3 [==============================] - 0s 7ms/step - loss: 0.5401 - accuracy: 0.6625


In [9]:
model.evaluate(X_test, Y_test, verbose=2)

2/2 - 0s - loss: 0.5655 - accuracy: 0.6857


[0.5655210018157959, 0.6857143044471741]